# Preliminaries

In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils import data
import wandb
import pandas as pd
from sklearn.model_selection import train_test_split
#wandb.init(project="HIV_kaggle")

Parse and look at first 5 rows

In [2]:
infineon = True # laptop
hpc_cluster = False

config = dict()

if infineon:
    config['path_workspace'] = 'C:\\Users\\SafontAndreu\\Workspace\\Visual Studio Code\\ps_hiv\\'
    config['path_inputs'] = 'C:\\Users\\SafontAndreu\\'
    
else:
    config['path_workspace'] = 'C:\\Users\\soren\\Alpen-Adria Universität Klagenfurt\\PhD - General\\Workspace\\Visual Studio Code\\'
    config['path_inputs'] = 'D:\\PhD\\'

config['path_database'] = config.get('path_workspace') + 'data\\'
config['path_models'] = config.get('path_inputs') + 'Models\\'
config['path_results'] = config.get('path_inputs') + 'Results\\'
config['path_ontology'] = config.get('path_inputs') + 'Ontology\\'

In [3]:
train_data = df = pd.read_csv(config.get('path_database')+'training_data.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
test_data = df = pd.read_csv(config.get('path_database')+'test_data_mod.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
sol_data = df = pd.read_csv(config.get('path_database')+'hivprogression_solution.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
train_data.head()

,PatientID,Resp,PR Seq,RT Seq,VL-t0,CD4-t0
0,1,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCCCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAGCTAAAGCCAGGAA...,4.3,145
1,2,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.6,224
2,3,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGGTAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.2,1017
3,4,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,5.7,206
4,5,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAGTAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.5,572


In [4]:
n_train = train_data.shape[0]

We have to remove the first two columns

In [5]:
all_features = train_data.iloc[:, 2:]
print(all_features.shape)
# one can assume if Seqs are not present it is a bad sign for survival
all_features["PR SeqNan"] = all_features["PR Seq"].apply(lambda x: pd.isna(x)).astype(bool)
all_features["RT SeqNan"] = all_features["RT Seq"].apply(lambda x: pd.isna(x)).astype(bool)
numeric_features = all_features.dtypes[(all_features.dtypes != 'object') & (all_features.dtypes != 'bool')].index
mean_numerical_features = all_features[numeric_features].mean()
std_numerical_features = all_features[numeric_features].std()
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / x.std() + 1e-4)
vt_mean = all_features["VL-t0"].mean()
cd4_mean = all_features["CD4-t0"].mean()
all_features["VL-t0"] = all_features["VL-t0"].fillna(vt_mean)
all_features["CD4-t0"] = all_features["CD4-t0"].fillna(cd4_mean)
all_features.head()


all_features['Resp'] = train_data['Resp']
print(all_features.shape)
print(all_features.head)

(1000, 4)
(1000, 7)
<bound method NDFrame.head of                                                 PR Seq  \
0    CCTCAAATCACTCTTTGGCAACGACCCCTCGTCCCAATAAGGATAG...   
1    CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGATAG...   
2    CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGGTAG...   
3    CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAGGATAG...   
4    CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAGTAAAGATAG...   
..                                                 ...   
995                                                NaN   
996                                                NaN   
997                                                NaN   
998                                                NaN   
999                                                NaN   

                                                RT Seq     VL-t0    CD4-t0  \
0    CCCATTAGTCCTATTGAAACTGTACCAGTAAAGCTAAAGCCAGGAA...  0.004059 -0.679249   
1    CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA... -0.989159 -0.280635   
2    CCCATTAGTCCTAT

# Tokenize and Vocab

In [6]:
import collections

def tokenize(seqs):
    return [tokenize_line(seq) for seq in seqs]

def tokenize_line(seq):
    if not pd.isna(seq):
        return list(seq)
    return []

class Vocab:
    def __init__(self, tokens):
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        self.idx_to_token = ['<unk>']
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): 
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [7]:
#X_name = 'PR Seq'
X_name = 'RT Seq'

y_name = 'Resp'

In [8]:
tokens_rt = tokenize(all_features[X_name].values)
vocab_rt = Vocab(tokens_rt)
list(vocab_rt.token_to_idx.items())

[('<unk>', 0),
 ('A', 1),
 ('T', 2),
 ('G', 3),
 ('C', 4),
 ('R', 5),
 ('Y', 6),
 ('M', 7),
 ('W', 8),
 ('K', 9),
 ('N', 10),
 ('S', 11),
 ('H', 12),
 ('D', 13),
 ('V', 14),
 ('B', 15)]

In [9]:
all_features[X_name] = all_features[X_name].apply(lambda x: vocab_rt[tokenize_line(x)])
all_features[X_name]

0      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
1      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
2      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
3      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
4      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
                             ...                        
995    [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
996    [4, 4, 4, 1, 2, 6, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
997    [4, 4, 4, 1, 2, 6, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
998    [4, 4, 2, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
999    [4, 4, 2, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
Name: RT Seq, Length: 1000, dtype: object

In [10]:
seq_pr = train_data['PR Seq']

seq_pr_unique = ''
for ele in seq_pr:
    if isinstance(ele, str):
        seq_pr_unique += ''.join(set(ele))
seq_pr_unique = ''.join(set(seq_pr_unique))
seq_pr_unique = ''.join(sorted(seq_pr_unique))
print('Unique nucleotides in PR Sequence = ', seq_pr_unique)



seq_rt = train_data['RT Seq']

seq_rt_unique = ''
for ele in seq_rt:
    if isinstance(ele, str):
        seq_rt_unique += ''.join(set(ele))
seq_rt_unique = ''.join(set(seq_rt_unique))
seq_rt_unique = ''.join(sorted(seq_rt_unique))
print('Unique nucleotides in RT Sequence = ', seq_rt_unique)

Unique nucleotides in PR Sequence =  ABCDGHKMNRSTVWY
Unique nucleotides in RT Sequence =  ABCDGHKMNRSTVWY


In [11]:
print(all_features.columns)

Index(['PR Seq', 'RT Seq', 'VL-t0', 'CD4-t0', 'PR SeqNan', 'RT SeqNan',
       'Resp'],
      dtype='object')


In [12]:
X = all_features[X_name].values
y = all_features[y_name].values

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

if X_name == 'PR Seq':
    vocab = seq_pr_unique
if X_name == 'RT Seq':
    vocab = seq_rt_unique

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, LSTM
from keras.losses import BinaryCrossentropy
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.preprocessing.sequence import pad_sequences

# Model configuration
additional_metrics = ['accuracy']
batch_size = 32
embedding_output_dims = 15
loss_function = BinaryCrossentropy()
max_sequence_length = 1005
num_distinct_words = len(vocab)+1
epochs = 5
lr = 2e-4
optimizer = adam_v2.Adam(learning_rate=lr, decay=lr/epochs)
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Pad all sequences
padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_valid = pad_sequences(x_valid, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

# Define the Keras model
model = Sequential()
model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
model.summary()

# Train the model
history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=epochs, verbose=verbosity_mode, validation_split=validation_split)

# Test the model after training
valid_results = model.evaluate(padded_inputs_valid, y_valid, verbose=False)
print(f'Validation results - Loss: {valid_results[0]} - Accuracy: {100*valid_results[1]}%')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1005, 15)          240       
                                                                 
 lstm (LSTM)                 (None, 10)                1040      
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 1,291
Trainable params: 1,291
Non-trainable params: 0
_________________________________________________________________
Train on 600 samples, validate on 150 samples
Epoch 1/5
600/600 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.4917

c:\Users\SafontAndreu\Workspace\Visual Studio Code\.venv\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


600/600 [==============================] - 9s 15ms/sample - loss: 0.6931 - accuracy: 0.4917 - val_loss: 0.6887 - val_accuracy: 0.7933
Epoch 2/5
600/600 [==============================] - 8s 14ms/sample - loss: 0.6842 - accuracy: 0.7983 - val_loss: 0.6798 - val_accuracy: 0.7933
Epoch 3/5
600/600 [==============================] - 8s 13ms/sample - loss: 0.6746 - accuracy: 0.7983 - val_loss: 0.6694 - val_accuracy: 0.7933
Epoch 4/5
600/600 [==============================] - 7s 12ms/sample - loss: 0.6631 - accuracy: 0.7983 - val_loss: 0.6567 - val_accuracy: 0.7933
Epoch 5/5
600/600 [==============================] - 7s 12ms/sample - loss: 0.6489 - accuracy: 0.7983 - val_loss: 0.6401 - val_accuracy: 0.7933
Validation results - Loss: 0.6418432645797729 - Accuracy: 78.39999794960022%


In [14]:
all_features_test = test_data.iloc[:, 2:]
print(all_features_test.shape)
# one can assume if Seqs are not present it is a bad sign for survival
all_features_test["PR SeqNan"] = all_features_test["PR Seq"].apply(lambda x: pd.isna(x)).astype(bool)
all_features_test["RT SeqNan"] = all_features_test["RT Seq"].apply(lambda x: pd.isna(x)).astype(bool)
numeric_features_test = all_features_test.dtypes[(all_features_test.dtypes != 'object') & (all_features_test.dtypes != 'bool')].index
mean_numerical_features_test = all_features_test[numeric_features_test].mean()
std_numerical_features_test = all_features_test[numeric_features_test].std()
all_features_test[numeric_features_test] = all_features_test[numeric_features_test].apply(lambda x: (x - x.mean()) / x.std() + 1e-4)
vt_mean_test = all_features_test["VL-t0"].mean()
cd4_mean_test = all_features_test["CD4-t0"].mean()
all_features_test["VL-t0"] = all_features_test["VL-t0"].fillna(vt_mean_test)
all_features_test["CD4-t0"] = all_features_test["CD4-t0"].fillna(cd4_mean_test)
all_features_test.head()


all_features_test['Resp'] = test_data['Resp']
print(all_features_test.shape)

(692, 4)
(692, 7)


In [15]:
tokens_pr_test = tokenize(all_features_test[X_name].values)
vocab_pr_test = Vocab(tokens_pr_test)
list(vocab_pr_test.token_to_idx.items())

[('<unk>', 0),
 ('A', 1),
 ('T', 2),
 ('G', 3),
 ('C', 4),
 ('R', 5),
 ('Y', 6),
 ('N', 7),
 ('M', 8),
 ('W', 9),
 ('K', 10),
 ('S', 11),
 ('D', 12),
 ('V', 13),
 ('H', 14),
 ('B', 15)]

In [16]:
all_features_test[X_name] = all_features_test[X_name].apply(lambda x: vocab_pr_test[tokenize_line(x)])
all_features_test[X_name]

0      [4, 4, 2, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
1      [4, 4, 4, 1, 2, 4, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
2      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
3      [4, 4, 2, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
4      [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
                             ...                        
687    [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
688    [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
689    [4, 4, 2, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
690    [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
691    [4, 4, 4, 1, 2, 2, 1, 3, 2, 4, 4, 2, 1, 2, 2, ...
Name: RT Seq, Length: 692, dtype: object

In [17]:
x_test = all_features_test[X_name]
y_test = all_features_test[y_name]

padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

Test results - Loss: 0.6980073717288199 - Accuracy: 50.0%
